# Preprocessing

In [6]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [7]:
ds = tfds.load("mnist", split="train")
ds = ds.map(lambda feature_dict: (feature_dict["image"], feature_dict["label"]))
ds = ds.map(lambda image, label: (tf.reshape(image, (-1,)), label))
ds = ds.map(lambda image, label: ((tf.cast(image, tf.float32)/128)-1, label))
ds = ds.map(lambda image, label: (image, tf.one_hot(label, depth=10)))
ds = ds.shuffle(2000).batch(200)
ds = ds.prefetch(4)

In [8]:
for elem in ds.take(1):
    print(elem)

(<tf.Tensor: shape=(200, 784), dtype=float32, numpy=
array([[-1., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1., -1., -1.],
       ...,
       [-1., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1., -1., -1.]], dtype=float32)>, <tf.Tensor: shape=(200, 10), dtype=float32, numpy=
array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)>)


2023-11-13 15:41:54.484133: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


# Model Creation via Subclassing from tf.keras.Model

In [9]:
class MLP_Model(tf.keras.Model):
    def __init__(self, layer_sizes, output_size=10):
        super().__init__()
        self.layers_list = []
        # layer_sizes e.g. [256,256]
        for layer_size in layer_sizes:
            new_layer = tf.keras.layers.Dense(units=layer_size, activation="sigmoid")
            self.layers_list.append(new_layer)
        self.output_layer = tf.keras.layers.Dense(units=output_size, activation="softmax")
        
    def call(self, x):
        for layer in self.layers_list:
            x = layer(x)
        y = self.output_layer(x)
        return y

# Training

In [14]:
model = MLP_Model(layer_sizes=[256,256])
cce = tf.keras.losses.CategoricalCrossentropy()
opt = tf.keras.optimizers.legacy.SGD()
ds = ds
EPOCHS=10

In [ ]:
import numpy as np
for epoch in range(EPOCHS):
    losses = []
    for x, target in ds:
        with tf.GradientTape() as tape: # context manager ('with' opens context)
            pred = model(x)
            loss = cce(y_true=target, y_pred=pred)
        grad = tape.gradient(loss, model.variables)
        opt.apply_gradients(zip(grad, model.variables))
        losses.append(loss.numpy())
    print(np.mean(losses))
